In [36]:
import torch
from torch_geometric.data import Data

In [35]:
!pip install torch_geometric

In [ ]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)

x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
data

Data(x=[3, 1], edge_index=[2, 4])

In [ ]:
data.validate(raise_on_error=True)

True

In [ ]:
data.num_edges

4

# Test mit einem Dataset

In [ ]:
from torch_geometric.datasets import TUDataset
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

In [ ]:
print(len(dataset))
print(dataset.num_classes)
print(dataset.num_node_features)

600
6
3


In [ ]:
data = dataset[50]
data

Data(edge_index=[2, 94], x=[27, 3], y=[1])

In [ ]:
dataset = dataset.shuffle()
train_dataset = dataset[:540]
test_dataset = dataset[540:]

In [ ]:
#batches mit dem torch_geometrix dataloader
from torch_geometric.loader import DataLoader

loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Simples Neural Net

In [ ]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch import nn

class GCN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = GCNConv(dataset.num_node_features, 16)
    self.conv2 = GCNConv(16, dataset.num_classes)

  def forward(self, data):
    x, edge_index = data.x, data.edge_index

    x = self.conv1(x, edge_index)
    x = F.relu(x)
    x = F.dropout(x, training=self.training)
    x = self.conv2(x, edge_index)
    return F.log_softmax(x, dim=1)

In [ ]:
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"
print(device)

#warschienlich sind imdataset meherere graphs enthalten
data = dataset[0].to(device)
print(data)

cpu
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [ ]:
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
  out = model(data)
  loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

In [ ]:
#test
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8150


# Jetzt auf einem anderen Dataset

In [37]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split
dataset = TUDataset(root='/path/to/your/dataset', name='ENZYMES')
train_size=400
val_size=100
test_size=100


In [74]:

torch.manual_seed(100)
train_data, temp_data = train_test_split(dataset, train_size=train_size, shuffle=True, random_state=42)
val_data, test_data = train_test_split(temp_data, train_size=val_size/(val_size+test_size), shuffle=True, random_state=42)


#making data set into batches
train_loader=DataLoader(dataset=train_data,batch_size=10,shuffle=True)
val_loader=DataLoader(dataset=val_data,batch_size=10,shuffle=False)

In [82]:
#glieche architektur?
import torch_geometric.nn as pyg_nn
import torch
from torch.nn import BatchNorm1d
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv
from torch import nn

if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"
print(device)

class GCN2(nn.Module):
  def __init__(self, num_features,hidden_size ,num_classes):
    super().__init__()
    self.conv1 = SAGEConv(num_features, hidden_size)
    self.conv2 = SAGEConv(hidden_size, hidden_size)
    self.conv3 = SAGEConv(hidden_size, hidden_size)
    self.conv4 = SAGEConv(hidden_size, hidden_size)

    self.pool = pyg_nn.global_mean_pool
    self.fc = nn.Linear(hidden_size, hidden_size)

    self.bn1 = BatchNorm1d(hidden_size)
    self.bn2 = BatchNorm1d(hidden_size)
    self.bn3 = BatchNorm1d(hidden_size)
    self.bn4 = BatchNorm1d(hidden_size)

  def forward(self, data):
    x, edge_index, batch = data.x, data.edge_index, data.batch
    x = self.conv1(x, edge_index)
    x = self.bn1(x)
    x = F.relu(x)
    #x = F.dropout(x, training=self.training)
    x = self.conv2(x, edge_index)
    x = self.bn2(x)
    x = F.relu(x)

    x = self.conv3(x, edge_index)
    x = self.bn3(x)
    x = F.relu(x)


    x = self.conv4(x, edge_index)
    x = self.bn4(x)
    x = F.relu(x)

    x = self.pool(x, batch)
    x = self.fc(x)

    return F.log_softmax(x, dim=1)

model = GCN2(num_features=3, hidden_size=40, num_classes=6).to(device)

cpu


In [83]:
d = next(iter(train_loader))
pred = model(d).argmax(dim=1)
pred

tensor([ 3,  1,  1,  1,  1,  3,  1,  1, 34, 22])

In [84]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

In [87]:
#training
model.train()
for epoch in range(200):
  total_loss = 0
  for data in train_loader:
    data = data.to(device)
    out = model(data)
    loss = loss_fn(out, data.y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch+1}/{epoch}, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1/0, Loss: 0.4277
Epoch 2/1, Loss: 0.4122
Epoch 3/2, Loss: 0.3136
Epoch 4/3, Loss: 0.3976
Epoch 5/4, Loss: 0.5023
Epoch 6/5, Loss: 0.4076
Epoch 7/6, Loss: 0.3505
Epoch 8/7, Loss: 0.3812
Epoch 9/8, Loss: 0.3848
Epoch 10/9, Loss: 0.3282
Epoch 11/10, Loss: 0.3440
Epoch 12/11, Loss: 0.3362
Epoch 13/12, Loss: 0.3005
Epoch 14/13, Loss: 0.3208
Epoch 15/14, Loss: 0.4016
Epoch 16/15, Loss: 0.4068
Epoch 17/16, Loss: 0.2810
Epoch 18/17, Loss: 0.2926
Epoch 19/18, Loss: 0.2964
Epoch 20/19, Loss: 0.3366
Epoch 21/20, Loss: 0.3704
Epoch 22/21, Loss: 0.2799
Epoch 23/22, Loss: 0.2385
Epoch 24/23, Loss: 0.2867
Epoch 25/24, Loss: 0.3387
Epoch 26/25, Loss: 0.3456
Epoch 27/26, Loss: 0.2903
Epoch 28/27, Loss: 0.3982
Epoch 29/28, Loss: 0.3071
Epoch 30/29, Loss: 0.3171
Epoch 31/30, Loss: 0.3228
Epoch 32/31, Loss: 0.2586
Epoch 33/32, Loss: 0.3289
Epoch 34/33, Loss: 0.3280
Epoch 35/34, Loss: 0.3782
Epoch 36/35, Loss: 0.2864
Epoch 37/36, Loss: 0.2737
Epoch 38/37, Loss: 0.2579
Epoch 39/38, Loss: 0.2330
Epoch

In [88]:
#evaluate
model.eval()
total_loss = 0

correct=0
total=0

with torch.no_grad():
  for data in val_loader:
    data = data.to(device)
    out = model(data)
    loss = loss_fn(out, data.y)
    total_loss += loss.item()

    _, predicted = torch.max(out, 1)
    total += data.y.size(0)
    correct += (predicted == data.y).sum().item()
correct/total

0.46